In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers

2023-08-12 13:48:40.515203: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-12 13:48:41.643701: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# CIFAR_10 is a set of 60K images 32x32 pixels on 3 channels
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32

#constant
BATCH_SIZE = 128
EPOCHS = 20
CLASSES = 10
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIM = tf.keras.optimizers.RMSprop()

2023-08-12 13:49:01.767230: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [3]:
#define the convnet 
def build(input_shape, classes):
    model = models.Sequential() 
    model.add(layers.Convolution2D(32, (3, 3), activation='relu',
                        input_shape=input_shape))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.25)) 

    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(classes, activation='softmax'))
    return model

In [4]:
# data: shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()

170498071/170498071 [==============================] - 733s 4us/step


In [5]:
# normalize
X_train, X_test = X_train / 255.0, X_test / 255.0

In [6]:
# convert to categorical
# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, CLASSES)
y_test = tf.keras.utils.to_categorical(y_test, CLASSES)

In [7]:
model=build((IMG_ROWS, IMG_COLS, IMG_CHANNELS), CLASSES)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 15, 15, 32)        0         
                                                                 
 flatten (Flatten)           (None, 7200)              0         
                                                                 
 dense (Dense)               (None, 512)               3686912   
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                        

In [8]:
# use TensorBoard, princess Aurora!
callbacks = [
  # Write TensorBoard logs to `./logs` directory
  tf.keras.callbacks.TensorBoard(log_dir='./logs')
]

In [9]:
# train
model.compile(loss='categorical_crossentropy', optimizer=OPTIM, metrics=['accuracy'])

In [10]:
model.fit(X_train, y_train, batch_size=BATCH_SIZE,
          epochs=EPOCHS, validation_split=VALIDATION_SPLIT, 
          verbose=VERBOSE, callbacks=callbacks)

Epoch 1/20


2023-08-12 14:02:18.382338: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 491520000 exceeds 10% of free system memory.


312/313 [============================>.] - ETA: 0s - loss: 1.7644 - accuracy: 0.3649

2023-08-12 14:02:41.528022: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 122880000 exceeds 10% of free system memory.


313/313 [==============================] - 24s 74ms/step - loss: 1.7639 - accuracy: 0.3649 - val_loss: 1.5223 - val_accuracy: 0.4619
Epoch 2/20
313/313 [==============================] - 23s 75ms/step - loss: 1.4451 - accuracy: 0.4871 - val_loss: 1.3057 - val_accuracy: 0.5475
Epoch 3/20
313/313 [==============================] - 26s 84ms/step - loss: 1.3067 - accuracy: 0.5390 - val_loss: 1.2683 - val_accuracy: 0.5468
Epoch 4/20
313/313 [==============================] - 25s 80ms/step - loss: 1.2106 - accuracy: 0.5756 - val_loss: 1.1941 - val_accuracy: 0.5857
Epoch 5/20
313/313 [==============================] - 25s 81ms/step - loss: 1.1362 - accuracy: 0.6018 - val_loss: 1.1103 - val_accuracy: 0.6182
Epoch 6/20
313/313 [==============================] - 26s 85ms/step - loss: 1.0684 - accuracy: 0.6274 - val_loss: 1.0754 - val_accuracy: 0.6352
Epoch 7/20
313/313 [==============================] - 26s 82ms/step - loss: 1.0124 - accuracy: 0.6461 - val_loss: 1.1377 - val_accuracy: 0.6097
Epo

In [11]:
score = model.evaluate(X_test, y_test,
                     batch_size=BATCH_SIZE, verbose=VERBOSE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

 4/79 [>.............................] - ETA: 1s - loss: 1.0845 - accuracy: 0.6523

2023-08-12 14:10:44.256440: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 122880000 exceeds 10% of free system memory.


79/79 [==============================] - 1s 18ms/step - loss: 1.0321 - accuracy: 0.6753

Test score: 1.0321269035339355
Test accuracy: 0.6753000020980835


In [12]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [13]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 391778), started 0:32:56 ago. (Use '!kill 391778' to kill it.)

# Improving the CIFAR-10 performance with a deeper network

In [14]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, regularizers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [21]:
def build_model(): 
    model = models.Sequential()
    
    #1st blocl
    model.add(layers.Conv2D(32, (3,3), padding='same', 
        input_shape=X_train.shape[1:], activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.2))

    #2nd block
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.3))

    #3d block 
    model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.4))

    #dense  
    model.add(layers.Flatten())
    model.add(layers.Dense(CLASSES, activation='softmax'))
    return model

    model.summary()

In [22]:
model = build_model()

In [23]:
model.compile(loss='categorical_crossentropy', 
            optimizer='RMSprop', 
            metrics=['accuracy'])

In [27]:
#train
batch_size = 64
model.fit(X_train, y_train, batch_size=BATCH_SIZE,
    epochs=10, validation_data=(X_test, y_test)) 

2023-08-12 14:14:17.580882: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 614400000 exceeds 10% of free system memory.


Epoch 1/10
391/391 [==============================] - ETA: 0s - loss: 1.8053 - accuracy: 0.4675

2023-08-12 14:17:12.689087: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 122880000 exceeds 10% of free system memory.


391/391 [==============================] - 182s 459ms/step - loss: 1.8053 - accuracy: 0.4675 - val_loss: 2.4182 - val_accuracy: 0.2827
Epoch 2/10
391/391 [==============================] - 178s 456ms/step - loss: 1.1755 - accuracy: 0.6411 - val_loss: 1.0403 - val_accuracy: 0.6777
Epoch 3/10
 32/391 [=>............................] - ETA: 2:34 - loss: 1.0214 - accuracy: 0.6838

KeyboardInterrupt: 

In [ ]:
score = model.evaluate(x_test, y_test,
                     batch_size=BATCH_SIZE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

# Improving the CIFAR-10 performance with data augmentation

In [ ]:
#image augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    )
datagen.fit(x_train)

In [ ]:
#train
batch_size = 64
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    epochs=10,
                    verbose=1,validation_data=(x_test,y_test))

In [28]:
#save to disk
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('model.h5')

In [30]:
#test
scores = model.evaluate(X_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0])) 

79/79 [==============================] - 8s 97ms/step - loss: 0.9635 - accuracy: 0.7055

Test result: 70.550 loss: 0.964


# Predicting with CIFAR-10

In [31]:
import numpy as np
#import scipy.misc

from skimage.transform import resize
from imageio import imread

from tensorflow.keras.models import model_from_json
from tensorflow.keras.optimizers import SGD

In [33]:
model_architecture = "model.json"
model_weights = "model.h5"
model = model_from_json(open(model_architecture).read())
model.load_weights(model_weights)

In [34]:
img_names = ["cat-standing.jpg", "dog.jpg"]
#imgs = [np.transpose(resize(imread(img_name), (32, 32)), (2, 0, 1)).astype("float32") 
#    for img_name in img_names]
imgs = [resize(imread(img_name), (32, 32)).astype("float32") for img_name in img_names]
imgs = np.array(imgs) / 255
print("imgs.shape:", imgs.shape)

/tmp/ipykernel_391989/1578175373.py:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  imgs = [resize(imread(img_name), (32, 32)).astype("float32") for img_name in img_names]


imgs.shape: (2, 32, 32, 3)


In [38]:
optim = SGD()
model.compile(loss="categorical_crossentropy", optimizer=optim, metrics=["accuracy"])

In [45]:
predictions = model.predict(imgs)
print("predictions:", np.argmax(predictions, axis=1))

1/1 [==============================] - 0s 33ms/step
predictions: [3 3]
